# ✋ Hand Gesture Controlled Brightness using OpenCV & MediaPipe

In this project, we use:
- **OpenCV** for webcam access and image processing.
- **MediaPipe Hands** for detecting and tracking hand landmarks.
- **NumPy** for mathematical operations.
- **screen-brightness-control (sbc)** to adjust the brightness of the system.

👉 The idea is simple:  
We measure the distance between the **thumb tip** and the **index finger tip**, and map that distance to the **screen brightness level**.


![My Image](media-pipe-hand.png)


## ✋ Step 1: Initialize MediaPipe Hands

- `mp.solutions.hands.Hands()` detects 21 landmarks for each detected hand.  
- `mp.solutions.drawing_utils` allows us to draw the landmarks and connections.


In [1]:
# Importing Libraries
import cv2
import mediapipe as mp
from math import hypot
import screen_brightness_control as sbc
import numpy as np

In [2]:
# Initializing the Model
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)

Draw = mp.solutions.drawing_utils

## 🎥 Step 2: Capture Video from Webcam

We use OpenCV's `cv2.VideoCapture(0)` to access the default webcam.  
- `0` means the first camera (laptop webcam).  
- `cap.read()` reads frames one by one.

---
## 🔄 Step 3: Process Frames in a Loop

- Each frame is read from the webcam.
- Flipped (`cv2.flip`) to behave like a mirror.
- Converted from **BGR → RGB** (MediaPipe requires RGB).

---
## 🖐 Step 4: Extract Hand Landmarks

- MediaPipe returns **21 hand landmarks** (normalized coordinates).
- We multiply by the **image width & height** to get pixel values.
- Save them in a list `landmarkList`.

---
## 👉 Step 5: Select Thumb & Index Finger Tips

- Landmark **4** = Thumb tip  
- Landmark **8** = Index finger tip  

We use these to measure distance and control brightness.

## 📏 Step 6: Measure Distance Between Fingers

We use the **Euclidean distance formula** (`hypot`) to measure the distance between thumb and index finger.

## 🌞 Step 7: Map Distance to Brightness

- Use `np.interp()` to map **distance range (15–220 px)** → **brightness range (0–100%)**.  
- Apply brightness using `sbc.set_brightness()`.

---
- This program detects your **hand landmarks** using MediaPipe.  
- Measures distance between **thumb tip (id=4)** and **index finger tip (id=8)**.  
- Maps this distance to **system brightness** (0–100%).  
- The result: Move fingers apart = brighter screen 🌞, move fingers close = dim screen 🌙.


In [3]:
# Start capturing video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read video frame by frame
    _,frame = cap.read()
    
    #Flip image 
    frame=cv2.flip(frame,1)
    
    # Convert BGR image to RGB image
    frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    # Process the RGB image
    Process = hands.process(frameRGB)
    
    landmarkList = []
    # if hands are present in image(frame)
    if Process.multi_hand_landmarks:
        # detect handmarks
        for handlm in Process.multi_hand_landmarks:
            for _id,landmarks in enumerate(handlm.landmark):
                # store height and width of image
                height,width,color_channels = frame.shape
                print(_id,landmarks)
                # calculate and append x, y coordinates
                # of handmarks from image(frame) to lmList
                x,y = int(landmarks.x*width),int(landmarks.y*height)              
                landmarkList.append([_id,x,y]) 
            
            # draw Landmarks
            Draw.draw_landmarks(frame,handlm,mpHands.HAND_CONNECTIONS)
    
    # If landmarks list is not empty
    if landmarkList != []:
        # store x,y coordinates of (tip of) thumb  
        x_1,y_1 = landmarkList[4][1],landmarkList[4][2]
        
        # store x,y coordinates of (tip of) index finger
        x_2,y_2 = landmarkList[8][1],landmarkList[8][2]
        
        # draw circle on thumb and index finger tip
        cv2.circle(frame,(x_1,y_1),7,(0,255,0),cv2.FILLED)
        cv2.circle(frame,(x_2,y_2),7,(0,0,255),cv2.FILLED)
        
        # draw line from tip of thumb to tip of index finger
        cv2.line(frame,(x_1,y_1),(x_2,y_2),(0,255,0),3)
        
        # calculate square root of the sum
        # of squares of the specified arguments.
        L = hypot(x_2-x_1,y_2-y_1)
        
        # 1-D linear interpolant to a function
        # with given discrete data points 
        # (Hand range 15 - 220, Brightness range 0 - 100),
        # evaluated at length.
        b_level = np.interp(L,[15,220],[0,100])
    
        # set brightness
        sbc.set_brightness(int(b_level))

    # Display Video and when 'q' is entered,
    # destroy the window
    cv2.imshow('Image', frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

0 x: 0.751030147
y: 0.952949882
z: -7.55694373e-007

1 x: 0.689260542
y: 0.94720459
z: -0.0147855403

2 x: 0.597899377
y: 0.872139454
z: -0.026725525

3 x: 0.540470779
y: 0.802465796
z: -0.043756526

4 x: 0.503893
y: 0.734041154
z: -0.0609776

5 x: 0.647491693
y: 0.689823151
z: -0.00735812495

6 x: 0.605568171
y: 0.595296919
z: -0.0451086909

7 x: 0.546120048
y: 0.587798238
z: -0.063353

8 x: 0.504594207
y: 0.610530317
z: -0.0720680803

9 x: 0.700572729
y: 0.67491138
z: -0.0236845035

10 x: 0.62351
y: 0.677647829
z: -0.0646915585

11 x: 0.628626
y: 0.792818666
z: -0.0609697215

12 x: 0.654972374
y: 0.82630682
z: -0.0469562933

13 x: 0.75626111
y: 0.68809396
z: -0.046398811

14 x: 0.677186489
y: 0.709173262
z: -0.0919353291

15 x: 0.684057474
y: 0.816557407
z: -0.0656992123

16 x: 0.712443471
y: 0.83787632
z: -0.0358761139

17 x: 0.810096
y: 0.718907714
z: -0.0711713806

18 x: 0.735179842
y: 0.731532693
z: -0.103186987

19 x: 0.730255425
y: 0.81378758
z: -0.0821873844

20 x: 0.758756816